# **SVM**

In [33]:
import pandas as pd
import sys
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, roc_auc_score, confusion_matrix, get_scorer, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
sys.path.append('..')
from utils import sensitivity_score, specificity_score

### **Data Preprocessing**

Import training and test dataset

In [34]:
train_data = pd.read_csv('/content/drive/MyDrive/dataset/train_data.csv')
train_labels = pd.read_csv('/content/drive/MyDrive/dataset/train_labels.csv')
test_data = pd.read_csv('/content/drive/MyDrive/dataset/test_data.csv')
test_labels = pd.read_csv('/content/drive/MyDrive/dataset/test_labels.csv')

In [35]:
from sklearn.model_selection import train_test_split

train_data, _, train_labels, _ = train_test_split(
    train_data, train_labels, test_size=0.8, stratify=train_labels, random_state=42
)

In [36]:
train_data.columns

Index(['_url', 'name', 'points', 'length', 'climb_total', 'profile',
       'startlist_quality', 'position', 'cyclist', 'cyclist_age', 'is_tarmac',
       'cyclist_team', 'duration', 'cyclist_number', 'cyclist_level',
       'cyclist_experience', 'relative_position', 'avg_relative_position',
       'cyclist_experience_profile', 'avg_rel_position_profile', 'length_cat',
       'cyclist_experience_length', 'avg_rel_position_length', 'climb_cat',
       'relative_position_sum', 'cyclist_experience_climb',
       'avg_rel_position_climb', 'avg_cyclist_level', 'position_entropy',
       'top_20_entropy'],
      dtype='object')

Define the categorical and numeric features. SVM requires only numeric features. Standardize the numeric features to ensure all variables have the same importance in distance calculations, as SVM is sensitive to differences in scale.

In [37]:
features_to_keep = [ 'cyclist_age', 'cyclist_number', 'cyclist_level', 'avg_relative_position',
       'avg_rel_position_profile', 'climb_total', 'cyclist_experience_climb',
       'avg_rel_position_climb', 'top_20_entropy']

train_data = train_data[features_to_keep]
test_data = test_data[features_to_keep]

numeric_transformer = StandardScaler()

In [38]:
# Create the preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[ ('num', numeric_transformer, features_to_keep)]
)

## **SVM**

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from imblearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, classification_report
from imblearn.under_sampling import RandomUnderSampler

# define the classifier
clf = SVC(class_weight='balanced', probability=False, random_state=42)

# create the pipeline
model = Pipeline(
    steps=[
        ("preprocessor", preprocessor),  
        ("sampler", RandomUnderSampler(random_state=42, sampling_strategy=0.5)),  
        ("svm", clf)                     
    ]
)

# svm hyperparameters
parameters = {
    'svm__C': [0.01, 0.05, 0.1, 1.0, 3.0, 5.0, 10.0],  # regularization parameter
    'svm__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  # kernel
    'svm__gamma': ['scale', 'auto', 0.1, 1],  
    'svm__degree': [2, 3, 5],  # only for poly kernel
}


scoring = {
    'sensitivity': make_scorer(sensitivity_score),
    'specificity': make_scorer(specificity_score),
    'accuracy': get_scorer("accuracy"),
    'precision': get_scorer("precision"),
    'sensitivity': get_scorer("recall"),
    'roc_auc': get_scorer("roc_auc"),
    'f1': get_scorer("f1")
}


rscv = RandomizedSearchCV(
    model,
    param_distributions=parameters,
    scoring=scoring,  
    n_iter=20,  
    cv=5,  
    verbose=1,
    n_jobs=-1,
    random_state=42,  
    refit="f1"  
)


rscv.fit(train_data, train_labels.values.ravel())

Fitting 5 folds for each of 20 candidates, totalling 100 fits


## Model Assessment

In [ ]:
# extract the results of the randomized search and best model idx
cv_results = rscv.cv_results_
best_model = rscv.best_estimator_
best_index = rscv.best_index_

#extract and print matrics
mean_test_scores = {metric: cv_results[f'mean_test_{metric}'][best_index] for metric in scoring.keys()}
std_test_scores = {metric: cv_results[f'std_test_{metric}'][best_index] for metric in scoring.keys()}

print("Validation results of the best model:")
for metric in scoring.keys():
    print(f"{metric.capitalize()} - Mean: {mean_test_scores[metric]:.4f}, Std: {std_test_scores[metric]:.4f}")



### Test scores:

In [ ]:
# Predict on the test data
test_predictions = rscv.predict(test_data)

# Calculate and visualize evaluation metrics
accuracy = round(accuracy_score(test_labels, test_predictions), 3)
recall = round(recall_with_zero_division(test_labels, test_predictions), 3)
precision = round(precision_with_zero_division(test_labels, test_predictions), 3)
sensitivity_score = round(sensitivity_score(test_labels, test_predictions), 3)
specificity_score = round(specificity_score(test_labels, test_predictions), 3)
f1 = round(f1_score(test_labels, test_predictions), 3)
roc_auc = round(roc_auc_score(test_labels, test_predictions), 3)
conf_matrix = confusion_matrix(test_labels, test_predictions)

print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"Sensitivity: {sensitivity_score}")
print(f"Specificity: {specificity_score}")
print(f"F1 Score: {f1}")
print(f"ROC AUC Score: {roc_auc}")

disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
disp.plot(cmap=plt.cm.Blues)
plt.show()


In [ ]:
report = classification_report(test_labels, test_predictions)
print(report)

In [ ]:
import joblib
import os

# Create the directory if it doesn't exist
os.makedirs('best_models', exist_ok=True)
# Save the best model
joblib.dump(best_model, 'best_models/rule_based.pkl')